In [1]:
import os
from functools import partial

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.contrib.slim as slim

from scipy.stats import poisson, norm

We change define the leaky_relu with default parameter `alpha=0.1`

In [2]:
leaky_relu = partial(tf.nn.leaky_relu,alpha=0.1)

Define parameters of the model.

In [3]:
batch_size = 64
learning_rate = 0.01
lambda_gradient = 0.025 # Gradient penalty
lambda_entropy = 3. # Entropy penalty
obs_alpha = 7. # Mean of first component of the simulator
prop_initial_mu = 0. # Initial mu of the proposal distribution 
prop_initial_log_sigma = 0. # Inintial log(sigma) of the proposal distribution
beta1 = 0.9
beta2 = 0.999
critic_steps = 100
count_steps = 500+1

#For reproducibility 
tf.set_random_seed(2210)
np.random.seed(2210)

Define simulator and generate observed samples.

In [4]:
#Define semipositive matrix as in paper
R = np.array([
    [1.31229955,  0.10499961,  0.48310515, -0.3249938,  -0.26387927],
    [0.10499961,  1.15833058, -0.55865473,  0.25275522, -0.39790775],
    [0.48310515, -0.55865473,  2.25874579, -0.52087938, -0.39271231],
    [0.3249938,   0.25275522, -0.52087938,  1.4034925,  -0.63521059],
    [-0.26387927, -0.39790775, -0.39271231, -0.63521059, 1.]])

# Define simulator
def simulator(alpha,beta):
    try:
        n = len(alpha)
    except TypeError:
        n = 1
    z0 = np.random.normal(loc=alpha.flatten())
    z1 = np.random.normal(loc=beta.flatten(),scale=3)
    z21 = np.random.normal(loc=-2,size=n)
    z22 = np.random.normal(loc=2,scale=0.5,size=n)
    r = np.random.choice([True, False], n)
    z2 = np.where(r, z21, z22)
    z3 = np.random.exponential(scale=3.,size=n)
    z4 = np.random.exponential(scale=0.5,size=n)
    return np.row_stack([z0,z1,z2,z3,z4]).T.dot(R)

#Generate real sample pool
N_obs = 200000
real_samples = simulator(np.array([N_obs*[1.]]).T,np.array([N_obs*[-1.]]).T)

Define AVO model.

In [5]:
proposal_shape = (batch_size,1)
batch_shape = (batch_size, 5)
critic_shape = (batch_size, 1)

# Placeholder for real distribution
X_real = tf.placeholder(tf.float32,shape=batch_shape)


# Define proposal parameters Psi for alpha (a) and beta (b)
with tf.variable_scope('proposal'):
    mu_a = tf.get_variable('mu_a', shape=(), initializer=tf.constant_initializer(prop_initial_mu))
    # We use logarithm of sigma since sigma >= 0 in order to stretch it over the real line
    lg_sigma_a = tf.get_variable('lg_sigma_a', shape=(), initializer=tf.constant_initializer(prop_initial_log_sigma))
    mu_b = tf.get_variable('mu_b', shape=(), initializer=tf.constant_initializer(prop_initial_mu))
    lg_sigma_b = tf.get_variable('lg_sigma_b', shape=(), initializer=tf.constant_initializer(prop_initial_log_sigma))

    
# Define proposal distribution q
proposal_distribution_a = tf.contrib.distributions.Normal(loc=mu_a, scale=tf.exp(lg_sigma_a))
proposal_distribution_b = tf.contrib.distributions.Normal(loc=mu_b, scale=tf.exp(lg_sigma_b))
sample_proposal_op_a = tf.stop_gradient(proposal_distribution_a.sample(sample_shape=proposal_shape))
sample_proposal_op_b = tf.stop_gradient(proposal_distribution_b.sample(sample_shape=proposal_shape))

# Compute the log probability for the parameters 
log_prob_prop = proposal_distribution_a.log_prob(sample_proposal_op_a)+\
                 proposal_distribution_b.log_prob(sample_proposal_op_b)
#Analytic differential entropy for the proposal distribution
entropy_proposal = lg_sigma_a+lg_sigma_b

# Placeholder for simulated distribution
X_sim = tf.placeholder(tf.float32,shape=batch_shape)

# Define the critic
def critic(x):
    layer = slim.stack(x, slim.fully_connected, [10, 10], scope='shared', activation_fn=leaky_relu)
    logits = slim.fully_connected(layer, 1, activation_fn=tf.identity, scope='logits')
    return logits

# Define interpolated data points for the Gradient Penalty Term
eps = tf.random_uniform(critic_shape, minval=0., maxval=1.)
X_interp = eps*X_real + (1.-eps)*X_sim

# Compute critic for different inputs sharing the same variables of the NN:
with tf.variable_scope('critic'):
        critic_real = critic(X_real)
        tf.get_variable_scope().reuse_variables()
        critic_sim = critic(X_sim)
        critic_interp = critic(X_interp)

# Gradient penalty 
grad = tf.gradients(critic_interp, [X_interp])[0]
gradient_penalty = lambda_gradient * tf.square(tf.norm(grad, 2,axis=1) - 1)

# Define losses
loss_critic = tf.reduce_mean(critic_sim - critic_real + gradient_penalty)
wgan_loss = -tf.reduce_mean(critic_sim - critic_real) # Distance between distributions
loss_prop = tf.reduce_mean(-tf.multiply(critic_sim, log_prob_prop) + lambda_entropy*entropy_proposal)

# Define trainable variables
prop_vars = [var for var in tf.trainable_variables() if var.name.startswith('proposal')]
critic_vars = [var for var in tf.trainable_variables() if var.name.startswith('critic')]

# Define optimizers
critic_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='critic')
prop_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='proposal')

# Define training operations
train_critic = critic_optimizer.minimize(loss_critic, var_list=critic_vars)
train_proposal = prop_optimizer.minimize(loss_prop, var_list=prop_vars)
# Define variables to reset for critic optimizer
velocity_vars = [critic_optimizer.get_slot(var, 'v') for var in critic_vars]
momentum_vars = [critic_optimizer.get_slot(var, 'm') for var in critic_vars]
reset_vel_mom = tf.variables_initializer(velocity_vars + momentum_vars)

#Reset beta power variables:
reset_b1 = tf.assign(critic_optimizer._beta1_power,beta1)
reset_b2 = tf.assign(critic_optimizer._beta2_power,beta2)
reset_critic = [reset_b1,reset_b2,reset_vel_mom]

Run AVO model to find the parameter distribution.

In [6]:
# Save losses
wgan_losses = np.zeros(count_steps)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    # Output the current values
    mu_a_val, lg_sigma_a_val, mu_b_val, lg_sigma_b_val = sess.run([mu_a, lg_sigma_a, mu_b, lg_sigma_b])
    print('Step 0: mu_a = {}, sigma_a = {}, mu_b = {}, sigma_b = {}'.format(mu_a_val, np.exp(lg_sigma_a_val),
                                                                            mu_b_val, np.exp(lg_sigma_b_val)))

    for epoch in range(count_steps):
        # Reset critic optimizer parameters at each epoch
        sess.run(reset_critic)

        # Optimize for the EM metric
        for idx in range(0, critic_steps):
            x_real = real_samples[np.random.choice(N_obs,size=batch_size)]
            alphas,betas = sess.run([sample_proposal_op_a,sample_proposal_op_b])
            x_sim = simulator(alphas,betas)
            _, this_loss = sess.run([train_critic, wgan_loss],
                                    feed_dict={X_real:x_real,
                                               X_sim:x_sim})
        # Save loss after optimizing
        wgan_losses[epoch] = this_loss

        # Update the proposal
        alphas,betas = sess.run([sample_proposal_op_a,sample_proposal_op_b])
        x_sim = simulator(alphas,betas)
        _ = sess.run(train_proposal,
                     feed_dict={X_sim:x_sim,
                                sample_proposal_op_a:alphas,
                                sample_proposal_op_b:betas})
        # Output the current values
        if epoch % 10 == 0:
            mu_a_val, lg_sigma_a_val, mu_b_val, lg_sigma_b_val = sess.run([mu_a, lg_sigma_a, mu_b, lg_sigma_b])
            print('Step {}: mu_a = {}, sigma_a = {}, mu_b = {}, sigma_b = {}, loss = {}'.format(epoch+1,mu_a_val, np.exp(lg_sigma_a_val),
                                                                                     mu_b_val, np.exp(lg_sigma_b_val), this_loss))

Step 0: mu_a = 0.0, sigma_a = 1.0, mu_b = 0.0, sigma_b = 1.0
Step 1: mu_a = 0.010000000707805157, sigma_a = 1.0100501775741577, mu_b = -0.009999999776482582, sigma_b = 1.0100501775741577, loss = 51.524932861328125
Step 11: mu_a = 0.08904185146093369, sigma_a = 1.0157819986343384, mu_b = -0.08384522795677185, sigma_b = 1.0554512739181519, loss = 90.0028076171875
Step 21: mu_a = 0.16901929676532745, sigma_a = 1.0073647499084473, mu_b = -0.16622188687324524, sigma_b = 1.0388904809951782, loss = 89.91385650634766
Step 31: mu_a = 0.25653719902038574, sigma_a = 0.9851034879684448, mu_b = -0.23532892763614655, sigma_b = 1.0244684219360352, loss = 58.791603088378906
Step 41: mu_a = 0.3377964496612549, sigma_a = 0.9695329666137695, mu_b = -0.27610480785369873, sigma_b = 1.023491621017456, loss = 67.99066162109375
Step 51: mu_a = 0.4062129557132721, sigma_a = 0.9522089958190918, mu_b = -0.32066935300827026, sigma_b = 1.017338514328003, loss = 26.9014835357666
Step 61: mu_a = 0.4565171003341675, 